## Initial setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
print(tf.__version__)
import torch
print(torch.__version__)
import matplotlib
print(matplotlib.__version__)

2.8.0
1.10.0+cu111
3.2.2


In [ ]:
!nvidia-smi

Fri Jan 21 19:41:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Other imports
! pip install tensorflow_addons
! pip install tensorflow_io

import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
from imutils import paths
from tqdm import tqdm
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import tensorflow_io as tfio
import tensorflow_hub as hub
import numpy as np
import cv2
import pandas as pd
import seaborn as sns
from scipy.stats import mannwhitneyu
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import sklearn.manifold
from sklearn.metrics.pairwise import cosine_similarity as cos
from sympy.utilities.iterables import multiset_permutations
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
from IPython.display import Image, display


import zipfile
import concurrent.futures

# Random seed fix
random_seed = 42
tf.random.set_seed(random_seed)
np.random.seed(random_seed)

     |████████████████████████████████| 1.1 MB 6.4 MB/s 
     |████████████████████████████████| 23.4 MB 1.8 MB/s 


## Dataset gathering and preparation

In [ ]:
%cd /content/drive/MyDrive/nanowire-morphology-classification-project

In [ ]:
training_batch_size = 4

BATCH_SIZE = training_batch_size

imageSize = 224

category_names = ['bundle', 'dispersed', 'network', 'singular']
color_method = ['C0', 'C1', 'C2', 'C3', 'C4']
color = ['black', 'magenta', 'cyan', 'yellow']
marker = ['o', 's', '<', '>', '^']
seaborn_palette = sns.color_palette("colorblind")

In [ ]:
# Image preprocessing utils
@tf.function
def parse_images(image_path):
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    # image = tfio.experimental.image.decode_tiff(image_string)[:, :, :3]   # in the doc, it transforms tiff to 4 channels, with additional channel of opacity which is not needed.
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[imageSize, imageSize])

    return image

## Initiate self-supervised models

In [ ]:
Resnet50_transfer = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(imageSize, imageSize, 3), 
    pooling=None,
)

Resnet50_transfer.trainable = False

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
# Resnet as backbone
def get_resnet_self_supervise_model(hidden_1, hidden_2, hidden_3):
    base_model = Resnet50_transfer
    base_model.trainable = True
    inputs = Input((imageSize, imageSize, 3))
    h = base_model(inputs, training=True)
    h = GlobalAveragePooling2D()(h)

    projection_1 = Dense(hidden_1)(h)                                        
    projection_1 = Activation("relu")(projection_1)
    projection_1 = BatchNormalization(epsilon=0.001)(projection_1)
    projection_2 = Dense(hidden_2)(projection_1)
    projection_2 = Activation("relu")(projection_2)
    projection_2 = BatchNormalization(epsilon=0.001)(projection_2)
    projection_3 = Dense(hidden_3)(projection_2)
    projection_3 = BatchNormalization(epsilon=0.001)(projection_3)

    resnet_model = Model(inputs, projection_3)
    
    return resnet_model

## segmentation data preparation, model and results

In [ ]:
np.random.seed(random_seed)
peptide_morph_seglabel_train_path = "/content/drive/MyDrive/TEM image datasets/2022-nanowire-morphology"
peptide_morph_images_train_seglabel = list(paths.list_files(basePath=peptide_morph_seglabel_train_path, validExts='png'))
peptide_morph_images_train_seglabel = np.random.choice(np.array(peptide_morph_images_train_seglabel), len(peptide_morph_images_train_seglabel), replace=False)
print(len(peptide_morph_images_train_seglabel))

200


In [ ]:
def generate_ground_truth_images(image, resolution):
  image_bool = np.ones((resolution, resolution))
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      if image[i, j, 1] == image[i, j, 2]:
        image_bool[i, j] = 0            # background is black with code of 0
      else: 
        image_bool[i, j] = 1            # nanowire is white with code of 1
  return image_bool

In [ ]:
segmentation_class_labels = []
for i in range(peptide_morph_images_train_seglabel.shape[0]):
  seg_class_label = peptide_morph_images_train_seglabel[i].split("/")[-2]
  segmentation_class_labels.append(seg_class_label)
le = LabelEncoder()
peptide_morph_train_seg_enc = le.fit_transform(segmentation_class_labels)

In [ ]:
# image_mask = np.zeros((len(peptide_morph_images_train_seglabel), imageSize, imageSize))
images_no_annotation_directory = []
images_no_annotation = np.zeros((len(peptide_morph_images_train_seglabel), imageSize, imageSize, 3))

for i in range(len(peptide_morph_images_train_seglabel)):
  images_no_annotation_directory.append('%s.jpg' % (peptide_morph_images_train_seglabel[i].split("_")[0] + "_" + peptide_morph_images_train_seglabel[i].split("_")[1]))
  images_no_annotation[i] = parse_images(images_no_annotation_directory[i])

  # these were used to create the ground truth grayscale images from the manual segmentation labels.
  # image_string = tf.io.read_file(peptide_morph_images_train_seglabel[i])
  # image = tf.image.decode_image(image_string, channels=3) / 255
  # image = tf.image.resize(image, (imageSize, imageSize))
  # image = tf.image.convert_image_dtype(image, tf.float32)
  # trans_nd_image_array = image.numpy()
  # image_mask[i] = generate_ground_truth_images(trans_nd_image_array, imageSize)
  
# np.savez_compressed('seg_mask_res512.npz', mask=image_mask)

# once we have the seg_mask saved, we can directly load from npz file
image_mask = np.load('seg_mask_res%i.npz' % (imageSize), allow_pickle=True)['mask']

In [ ]:
def upsample(filters, size, apply_dropout=False):

  result = Sequential()
  result.add(
    Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer='he_uniform',
                                    use_bias=False))

  result.add(BatchNormalization())

  if apply_dropout:
      result.add(Dropout(0.5))

  result.add(ReLU())

  return result

In [ ]:
# build our resnet50-unet model with the option of transfered weights from either tem images
def unet_model(base_model, resolution):
  inputs = tf.keras.layers.Input(shape=[resolution, resolution, 3])

  # Use the activations of these layers
  layer_names = [
      'conv1_relu',         # (batch_size, 112, 112, 64)
      'conv2_block3_out',   # (batch_size, 56, 56, 256)
      'conv3_block4_out',   # (batch_size, 28, 28, 512)
      'conv4_block6_out',   # (batch_size, 14, 14, 1024)
      'conv5_block3_out'    # (batch_size, 7, 7, 2048)
  ]
  base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

  # Create the feature extraction model
  encoder = Model(inputs=base_model.input, outputs=base_model_outputs)

  encoder.trainable = False

  decoder = [
    upsample(1024, 4),  # (batch_size, 7, 7, 2048)
    upsample(512, 4),  # (batch_size, 14, 14, 1024)
    upsample(256, 4),  # (batch_size, 28, 28, 512)
    upsample(128, 4),  # (batch_size, 56, 56, 256)
  ]

  last = Conv2DTranspose(1, 4,    # number of class, size of input vector (batch_size, x, y, class)
                              strides=2,
                              padding='same',
                              use_bias=True,
                              kernel_initializer='he_uniform',
                              activation='sigmoid')       # (batch_size, resolution, resolution, 3)

  # Downsampling through the model
  skips = encoder(inputs)
  x = skips[-1]
  skips = reversed(skips[:-1])        # exclude the last conv layer, reverse is a python native function

  # Upsampling and establishing the skip connections
  for up, skip in zip(decoder, skips):
    x = up(x)
    x = Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
# define loss functions and metrics for segmentation task
# coef functions are for batches of images, score functions are for a single image

def dice_coef(y_true, y_pred, smooth=0.000001):

    beta = 1
    tp = tf.reduce_sum(y_true * y_pred, axis=[1,2])
    fp = tf.reduce_sum(y_pred, axis=[1,2]) - tp
    fn = tf.reduce_sum(y_true, axis=[1,2]) - tp

    score = tf.reduce_mean(((1 + beta ** 2) * tp + smooth) \
            / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + smooth))
    return score

def dice_score(y_true, y_pred, smooth=0.000001):
  y_true = y_true.astype('float32')
  beta = 1
  tp = tf.reduce_sum(y_true * y_pred, axis=[1])
  fp = tf.reduce_sum(y_pred, axis=[1]) - tp
  fn = tf.reduce_sum(y_true, axis=[1]) - tp

  score = tf.reduce_mean(((1 + beta ** 2) * tp + smooth) \
          / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + smooth)).numpy()
  return score

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def iou_coef(y_true, y_pred, smooth=0.000001):

    beta = 1
    tp = tf.reduce_sum(y_true * y_pred, axis=[1,2])
    fp = tf.reduce_sum(y_pred, axis=[1,2]) - tp
    fn = tf.reduce_sum(y_true, axis=[1,2]) - tp

    score = tf.reduce_mean((tp + smooth) \
            / (tp + fn + fp + smooth))
    return score

def iou_score(y_true, y_pred, smooth=0.000001):
  y_true = y_true.astype('float32')
  tp = tf.reduce_sum(y_true * y_pred, axis=[1])
  fp = tf.reduce_sum(y_pred, axis=[1]) - tp
  fn = tf.reduce_sum(y_true, axis=[1]) - tp

  score = tf.reduce_mean((tp + smooth) \
          / (tp + fn + fp + smooth)).numpy()
  return score

def iou_coef_loss(y_true, y_pred):
    return 1 - iou_coef(y_true, y_pred)

In [ ]:
tf.random.set_seed(random_seed)

encoder_res = np.array([224, 384, 224])
encoder_batch_size = np.array([64, 16, 64])
random_seed_list = np.array([45])
# random_seed_for_split = np.linspace(42, 42 + 3, 4).astype(int)
random_seed_for_split = np.array([42])
training_image_size = np.array([80, 32, 16, 8, 4, 2, 1])
n_epoch = np.array([20, 30, 40, 60, 120, 120, 120])

for i in range(len(random_seed_list)):
  resnet_model = get_resnet_self_supervise_model(128, 64, 1024)
  if encoder_res[0] == 224:
    resnet_model.load_weights('barlow_resnet_batch%i_project128_64_1024_seed%i.h5' % (encoder_batch_size[0], random_seed_list[i]))
  else:
    resnet_model.load_weights('barlow_resnet_batch%i_project128_64_1024_res%i_seed%i.h5' % (encoder_batch_size[0], encoder_res[0], random_seed_list[i]))
  trained_resnet50 = resnet_model.get_layer('resnet50')

  for j in range(len(random_seed_for_split)):
    seg_scores = np.zeros((len(training_image_size), int(image_mask.shape[0] / 5), 2))

    X_TRAIN, x_val, MASK_TRAIN, mask_val = train_test_split(images_no_annotation, image_mask, test_size=0.2, shuffle=True, stratify=peptide_morph_train_seg_enc, random_state=random_seed_for_split[j])
    X_TRAIN, x_val, TRAIN_FILENAME, test_filename = train_test_split(images_no_annotation, peptide_morph_images_train_seglabel, test_size=0.2, shuffle=True, stratify=peptide_morph_train_seg_enc, random_state=random_seed_for_split[j])
    X_TRAIN, x_val, seg_class_train, seg_class_val = train_test_split(images_no_annotation, peptide_morph_train_seg_enc, test_size=0.2, shuffle=True, stratify=peptide_morph_train_seg_enc, random_state=random_seed_for_split[j])      

    for n in range(len(training_image_size)):
      if os.path.isdir('segmentation_figures/res%i_enc_res%i_seed%i_seed%i_datasize%i' %(imageSize, encoder_res[0], random_seed_list[i], random_seed_for_split[j], training_image_size[n])) != True:
        os.mkdir('segmentation_figures/res%i_enc_res%i_seed%i_seed%i_datasize%i' %(imageSize, encoder_res[0], random_seed_list[i], random_seed_for_split[j], training_image_size[n]))

      if training_image_size[n] != 80:
        x_train, x_no_val, mask_train, mask_no_val = train_test_split(X_TRAIN, MASK_TRAIN, test_size=(1 - training_image_size[n] * 2 / len(seg_class_train)), shuffle=True, stratify=seg_class_train, random_state=42)
        x_train, x_no_val, train_filename, filename_no_val = train_test_split(X_TRAIN, TRAIN_FILENAME, test_size=(1 - training_image_size[n] * 2 / len(seg_class_train)), shuffle=True, stratify=seg_class_train, random_state=42)
      else:
        x_train = X_TRAIN
        mask_train = MASK_TRAIN

      # train segmentation model
      unet = unet_model(trained_resnet50, imageSize)
      unet.compile(optimizer='adam',
                    loss=dice_coef_loss,
                    metrics=[iou_coef, dice_coef])
      model_history = unet.fit(x=x_train, y=mask_train, epochs=n_epoch[n],
                                batch_size=training_batch_size,
                                validation_data=(x_val, mask_val)
                                )
      # log best segmentation model performance
      y_predict = unet.predict(x_val, batch_size=training_batch_size)
      y_predict = y_predict.reshape((len(x_val), imageSize, imageSize))

      filename = []
      for k in range(y_predict.shape[0]):
        filename.append(test_filename[k].split("/")[-1])
        seg_scores[n, k, 0] = iou_score(mask_val[k], y_predict[k])
        seg_scores[n, k, 1] = dice_score(mask_val[k], y_predict[k])
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,20))
        ax1.imshow(x_val[k])
        nanowire_pix = np.transpose(np.vstack((np.where(mask_val[k] == 0)[0], np.where(mask_val[k] == 0)[1])))
        image_a = np.expand_dims(mask_val[k], axis=-1) * np.array([1, 1, 1])
        ax2.imshow(image_a)
        nanowire_pix = np.transpose(np.vstack((np.where(y_predict[k] == 0)[0], np.where(y_predict[k] == 0)[1])))
        image_a = np.expand_dims(y_predict[k], axis=-1) * np.array([1, 1, 1])
        ax3.imshow(image_a)
        plt.title('iou: %1.4f dice: %1.4f' % (seg_scores[n, k, 0], seg_scores[n, k, 1]))
        plt.savefig('segmentation_figures/res%i_enc_res%i_seed%i_seed%i_datasize%i/%s' %(imageSize, encoder_res[0], random_seed_list[i], random_seed_for_split[j], training_image_size[n], test_filename[k].split("/")[-1]))
        plt.close('all')

    np.savez_compressed('segmentation_scores_main_result/res%i_enc_res%i_seed%i_seed%i.npz' %(imageSize, encoder_res[0], random_seed_list[i], random_seed_for_split[j]), scores=seg_scores, filename=filename)

Epoch 1/20
40/40 [==============================] - 18s 343ms/step - loss: 0.5349 - iou_coef: 0.3158 - dice_coef: 0.4651 - val_loss: 0.6011 - val_iou_coef: 0.2629 - val_dice_coef: 0.3989
Epoch 2/20
40/40 [==============================] - 12s 305ms/step - loss: 0.4216 - iou_coef: 0.4197 - dice_coef: 0.5784 - val_loss: 0.5148 - val_iou_coef: 0.3338 - val_dice_coef: 0.4852
Epoch 3/20
40/40 [==============================] - 12s 306ms/step - loss: 0.3387 - iou_coef: 0.5071 - dice_coef: 0.6613 - val_loss: 0.5365 - val_iou_coef: 0.3160 - val_dice_coef: 0.4635
Epoch 4/20
40/40 [==============================] - 12s 307ms/step - loss: 0.2714 - iou_coef: 0.5817 - dice_coef: 0.7286 - val_loss: 0.5199 - val_iou_coef: 0.3307 - val_dice_coef: 0.4801
Epoch 5/20
40/40 [==============================] - 12s 307ms/step - loss: 0.2524 - iou_coef: 0.6036 - dice_coef: 0.7476 - val_loss: 0.3672 - val_iou_coef: 0.4770 - val_dice_coef: 0.6328
Epoch 6/20
40/40 [==============================] - 12s 307ms/ste